In [2]:
import json
from config import config
from pathlib import Path
import pandas as pd
from collections.abc import Generator
from typing import Any, TypeAlias

In [ ]:
RAW_FORMULA1_COMMENTS_FILE = config.RAW_DATA_DIR / 'formula1_comments.ndjson'
RAW_FORMULA1_SUBMISSIONS_FILE = config.RAW_DATA_DIR / 'formula1_submissions.ndjson'
RAW_FORMULA1POINT5_COMMENTS_FILE = config.RAW_DATA_DIR / 'formula1point5_comments.ndjson'
RAW_FORMULA1POINT5_SUBMISSIONS_FILE = config.RAW_DATA_DIR / 'formula1point5_submissions.ndjson'

CommentsDict: TypeAlias = dict[str, Any]
SubmissionsDict: TypeAlias = dict[str, Any]

def load_ndjson(ndjson_file: Path) -> Generator[dict[str, Any]]:
    with open(ndjson_file, 'r', encoding='utf-8') as file:
        for line in file:
            yield json.loads(line)

def load_comments(ndjson_file: Path) -> Generator[CommentsDict]:
    yield from load_ndjson(ndjson_file)

def load_submissions(ndjson_file: Path) -> Generator[SubmissionsDict]:
    yield from load_ndjson(ndjson_file)

In [ ]:
display(tuple(load_submissions(RAW_FORMULA1POINT5_SUBMISSIONS_FILE)))



In [ ]:
formula1_comments = tuple(load_comments(RAW_FORMULA1_COMMENTS_FILE))
formula1_submissions = tuple(load_submissions(RAW_FORMULA1_SUBMISSIONS_FILE))
formula1point5_comments = tuple(load_comments(RAW_FORMULA1POINT5_COMMENTS_FILE))
formula1point5_submissions = tuple(load_submissions(RAW_FORMULA1POINT5_SUBMISSIONS_FILE))